US Consumer Behaviour 

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
pce = pd.read_csv("../data/PERSONAL_EXPENDITURE.csv")
save = pd.read_csv("../data/SAVING_RATE.csv")
credit = pd.read_csv("../data/CREDIT_OWNED.csv")
cpi = pd.read_csv("../data/cpiaucsl.csv")

In [13]:
for df in [credit, save, pce, cpi]:
    df.rename(columns={'observation_date': 'DATE'}, inplace=True)
    df['DATE'] = pd.to_datetime(df['DATE'])

In [14]:
master_df = pce.merge(save, on='DATE', how='inner') \
                .merge(credit, on='DATE', how='inner')\
                .merge(cpi, on='DATE', how='inner')

In [15]:
master_df.rename(columns={
    'PCEC96': 'Expenditure_Billions',
    'PSAVERT': 'Saving_Rate_Pct',
    'TOTALSL': 'Credit_Owned_Billions',
    'CPIAUCSL': 'CPI_Index'
}, inplace=True)

In [16]:
master_df.sort_values('DATE', inplace=True)

In [17]:
master_df['Credit_to_Spend_Ratio'] = master_df['Credit_Owned_Billions'] / master_df['Expenditure_Billions']

master_df['Spend_YoY_Growth'] = master_df['Expenditure_Billions'].pct_change(periods=12) * 100
master_df['Credit_YoY_Growth'] = master_df['Credit_Owned_Billions'].pct_change(periods=12) * 100
master_df['Inflation_YoY'] = master_df['CPI_Index'].pct_change(periods=12) * 100

def assign_regime(date):
    if date < pd.to_datetime('2020-03-01'):
        return '1_Pre_Covid'
    elif date < pd.to_datetime('2021-06-01'):
        return '2_Covid_Stimulus'
    else:
        return '3_Post_Inflation_Shock'

master_df['Regime'] = master_df['DATE'].apply(assign_regime)


master_df.dropna(inplace=True)


print(master_df)

          DATE  Expenditure_Billions  Saving_Rate_Pct  Credit_Owned_Billions  \
12  2008-01-01               11333.2              2.6             2619427.65   
13  2008-02-01               11293.9              3.0             2634496.42   
14  2008-03-01               11322.1              2.9             2645603.64   
15  2008-04-01               11340.5              2.4             2654243.23   
16  2008-05-01               11361.6              6.8             2660193.15   
..         ...                   ...              ...                    ...   
221 2025-06-01               16466.3              4.6             5042689.74   
222 2025-07-01               16547.5              4.3             5057212.70   
223 2025-08-01               16595.8              4.1             5059896.38   
224 2025-09-01               16614.3              4.0             5071365.99   
226 2025-11-01               16715.4              3.5             5084831.24   

     CPI_Index  Credit_to_Spend_Ratio  

C:\Users\tbbro\AppData\Local\Temp\ipykernel_7536\974551343.py:5: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  master_df['Inflation_YoY'] = master_df['CPI_Index'].pct_change(periods=12) * 100


In [18]:
# Save master_df to CSV
master_df.to_csv('../data/master_df.csv', index=False)
print('Saved ../data/master_df.csv')

Saved ../data/master_df.csv
